# DNBLab Jupyter Notebook Skript 

## Abfrage der SRU-Schnittstelle



### Importieren passender Bibliotheken  <a class="anchor" id="Teil1"></a>

In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

### Schnelle Abfrage (100 erste Treffer, nur Rückgabe der Treffermenge)  <a class="anchor" id="Teil1"></a>

In [2]:
def sru_dnb_simple(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'MARC21-xml', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    number = response.find_all('numberofrecords')[0].text
    
    return number
    
    

In [3]:
myquery1 = sru_dnb_simple('HSS=diss* and (sgt="3*" or sgt="5*" or sgt="6*") and jhr="1980"')
print(myquery1, "Ergebnisse")

15754 Ergebnisse


### Abfrage aller Ergebnisse über SRU - ACHTUNG: Das kann dauern! 

Bei sehr umfangreichen Anfragen kann es ggf. zu Abbrüchen kommen - in diesem Fall bietet es sich an, die Abfrage entweder noch einmal aufzuteilen oder bspw. nach je 100 Schliefendurchläufen eine Pause einzubauen. 

In [7]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records



In [8]:
myquery = dnb_sru('HSS=diss* and (sgt="3*" or sgt="5*" or sgt="6*") and jhr="1980"')
print(len(myquery), "Ergebnisse")

15754 Ergebnisse


In [75]:
# Ausgabe EINES Treffers aus der XML-Antwort der Schnittstelle 
# Die Zahl in Klammern gibt die Nummer des angezeigten Treffers wieder, beginnend bei 0 für den ersten Treffer. 

print(myquery[0])

<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
<leader>00000nam a2200000uc 4500</leader>
<controlfield tag="001">1253067287</controlfield>
<controlfield tag="003">DE-101</controlfield>
<controlfield tag="005">20220308223115.0</controlfield>
<controlfield tag="007">cr||||||||||||</controlfield>
<controlfield tag="008">220308s2022    gw |||||om||| 00||||eng  </controlfield>
<datafield ind1=" " ind2=" " tag="015">
<subfield code="a">22,O04</subfield>
<subfield code="2">dnb</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="016">
<subfield code="2">DE-101</subfield>
<subfield code="a">1253067287</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="024">
<subfield code="2">urn</subfield>
<subfield code="a">urn:nbn:de:bsz:21-dspace-994892</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="035">
<subfield code="a">(DE-599)DNB1253067287</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="040">
<subfield code="a">1240</subfield>
<subfield code="b">

### Verarbeiten der Ergebnisse  <a class="anchor" id="Teil4"></a>

Hier wird die XML-Antwort der SRU-Schnittstelle Treffer für Treffer übergeben und bestimmte Inhalte herausgezogen, um im Anschluss in ein Dataframe zur weiteren Arbeit und Analyse übertragen zu werden.

In [9]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unknown"
        
    
    # author
    author = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        author = author[0].text
    except:
        author = "unknown"
        
        
    #gnd
    gnd = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = '0']", namespaces=ns)
    try:
        gnd = gnd[0].text
    except:
        gnd = "none"
  

    # supervisor (maybe):
    superv = xml.xpath("marc:datafield[@tag = '700']/marc:subfield[@code = 'a']", namespaces=ns)
    superrole = xml.xpath("marc:datafield[@tag = '700']/marc:subfield[@code = 'e']", namespaces=ns)
    try:
        superv = superv[0].text
        superrole = superrole[0].text
        supervisor = superv + " - " + superrole
    except:
        supervisor = "unknown"
    
    
    meta_dict = {"idn":idn, "titel":titel, "author":author, "gnd":gnd, "supervisor":supervisor}
    
    return meta_dict



In [10]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,idn,titel,author,gnd,supervisor
0,801327865,1-O-Hydroxicinnamyl-β-D-glucopyranosen [OHyd...,"Reichert, Angelika",none,unknown
1,820259373,10-Jahres-Statistik [Zehn-Jahres-Statistik] ...,"Charlier, Thomas",none,unknown
2,810613190,10 [Zehn] Serumproteine bei gesunden HBs-Ag...,"Mader, Gudrun",(DE-588)1158850506,unknown
3,213196441,10[Zehn]-Jahres-Statistik der Sectiones caesar...,"Charlier, Thomas",none,unknown
4,810017954,"1,1-Di-tert-butyl-1-silacyclobutan [Ditertbu...","Rankers, Reinhard",(DE-588)1042386145,unknown
...,...,...,...,...,...
15749,830821406,Zytogenetische Untersuchungen bei Patienten mi...,"Mayer, Claudia Ulrike",none,unknown
15750,820344974,Zytologische und zytochemische Untersuchungen ...,"Schulz, Dieter",none,unknown
15751,840858868,Zytophotometrisch ermittelter Kern-DNS-Gehalt ...,"Hinz, Barbara",none,unknown
15752,820961787,Zytophotometrische DNS-Messungen an Hodgkin- u...,"Zugmaier, Gerhard",none,unknown


In [115]:
#Speichern der Ergebnisse aus dem Dataframe als CSV-Datei: 
df.to_csv("Abfrage.csv", index=False) 

In [122]:
#Speichern der Ergebnisse aus dem Dataframe als Excel-Datei: 
df.to_excel("Abfrage.xlsx", index=False, encoding='utf8') 

### Filtern des Dataframe

In [11]:
## Show all items where no author was found: 

noauthors = df.query('author == "unknown"')
noauthors

,idn,titel,author,gnd,supervisor
963,810363992,"Beiträge zur ""Energetischen Geomorphologie"" in...",unknown,none,"Hempel, Ludwig - Verfasser"
1592,208938184,Biostratigraphie und Ammonitenfauna des Unter-...,unknown,none,unknown
2524,810964007,Der Einfluss des Sauerstoffpartialdrucks in ...,unknown,none,"Bondke, Hansjürgen - Mitwirkender"
2658,800013018,Der Entwurf einer Polizeiordnung für das Her...,unknown,none,"Fronzek, Brigitte - Mitwirkender"
2914,830662197,Deutsche Soldatenlieder,unknown,none,unknown
3780,801140455,Die Handschriften der Sammlung J 1 im Haupts...,unknown,none,unknown
3822,801112494,Die Inschriften von Sestos und der thrakisch...,unknown,none,"Krauss, Johannes - Herausgeber"
9503,801098114,Planungsgrundlagen für eine gemeindenahe psych...,unknown,none,"Lehmkuhl, Dieter - Mitwirkender"
9693,213230720,Primärstruktur des Polypeptids IV aus Cytochro...,unknown,none,unknown
10540,810300346,Simplexoptimierung fluorimetrischer Aktivitäts...,unknown,none,unknown
